<a href="https://colab.research.google.com/github/RicardoVilla0/MachineLearning/blob/main/Unidad%203/proyecto/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Importamos librerias necesarias para knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

In [2]:
prestamos_df = pd.read_csv('prestamos_ok.csv')
prestamos_df.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,issue_year,repaid,loan_term_year,purpose_code,home_ownership_code,grade_code,addr_state_code
0,0,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,Cash,N,NaN,2011,1,3,11,4,2,2
1,1,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,Cash,N,NaN,2011,1,3,9,4,2,0
2,2,3000,3000,3000.0,36 months,18.64,109.43,E,E1,MKC Accounting,...,Cash,N,NaN,2011,1,3,0,4,4,0
3,3,5600,5600,5600.0,60 months,21.28,152.39,F,F2,NaN,...,Cash,N,NaN,2011,0,5,11,3,5,0
4,4,5375,5375,5350.0,60 months,12.69,121.45,B,B5,Starbucks,...,Cash,N,NaN,2011,0,5,9,4,1,5


In [3]:
X = prestamos_df[['loan_amnt', 'int_rate', 'annual_inc',
                  'dti', 'purpose_code','grade_code']]

# Variable objetivo o variable a predecir
y = prestamos_df["repaid"]

In [4]:
# Dividimos el dataFrame df en df_train y df_test.
# 70% para dataset de entrenamient0, y 30% para dataset de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# verificamos la cantidad de registros asignados al dataframe de entrenamiento
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13935, 6), (5973, 6), (13935,), (5973,))

In [5]:
from sklearn.preprocessing import StandardScaler

# Normalizar variables (KNN es sensible a la escala)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Entrenar KNN
knn_scaled = KNeighborsClassifier(n_neighbors=1)
knn_scaled.fit(X_train_scaled, y_train)

y_pred_scaled = knn_scaled.predict(X_test_scaled)
print("\nReporte de clasificación datos normalizados y 10 vecinos:")
print(classification_report(y_test, y_pred_scaled, target_names=["No Pagado", "Pagado"]))

print ("\nMatriz de confusión con datos normalizados y 10 vecinos:\n", confusion_matrix(y_test, y_pred_scaled), "\n")


Reporte de clasificación datos normalizados y 10 vecinos:
              precision    recall  f1-score   support

   No Pagado       0.22      0.20      0.21       930
      Pagado       0.86      0.87      0.86      5043

    accuracy                           0.77      5973
   macro avg       0.54      0.54      0.54      5973
weighted avg       0.76      0.77      0.76      5973


Matriz de confusión con datos normalizados y 10 vecinos:
 [[ 185  745]
 [ 640 4403]] 



In [7]:
# Creamoos el clasificador de regresion logistica
knn10 = KNeighborsClassifier(n_neighbors = 10)

# Entrenamos el clasificador
knn10.fit(X_train, y_train)

# Precisión del modelo en la fase de entrenamiento
print("Precision del clasificador en fase de entrenamiento", knn10.score(X_train, y_train) )


Precision del clasificador en fase de entrenamiento 0.8553283100107643


In [8]:
from sklearn.inspection import permutation_importance

result = permutation_importance(knn10, X_test, y_test, n_repeats=30, random_state=42)

importancia = pd.DataFrame({
    "Variable": X.columns,
    "Importancia Media": result.importances_mean,
    "Desviación": result.importances_std
}).sort_values("Importancia Media", ascending=False)

print(importancia)

       Variable  Importancia Media  Desviación
2    annual_inc           0.004001    0.001194
0     loan_amnt           0.003365    0.001417
3           dti           0.000960    0.000645
1      int_rate           0.000028    0.000430
5    grade_code          -0.000078    0.000219
4  purpose_code          -0.000234    0.000373


In [9]:
# Visualizar importancias con Plotly
import plotly.express as px
fig4 = px.bar(importancia, x="Variable", y="Importancia Media",
              error_y="Desviación", title="Importancia de características (KNN - Permutation Importance)",
              text_auto=".3f", color="Variable")
fig4.update_layout(width=800, height=600)
fig4.show()

In [10]:
from sklearn.preprocessing import StandardScaler

X = prestamos_df[['loan_amnt', 'int_rate', 'annual_inc',
                  'dti', 'purpose_code','grade_code']]

y = prestamos_df["repaid"]

# Dividir Datos, stratify es para que mantenga la misma proporción de clases en ambos conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Normalizar variables (KNN es sensible a la escala)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# Visualizar importancias con Plotly
import plotly.express as px
fig4 = px.bar(importancia, x="Variable", y="Importancia Media",
              error_y="Desviación", title="Importancia de características (KNN - Permutation Importance)",
              text_auto=".3f", color="Variable")
fig4.update_layout(width=800, height=600)
fig4.show()

In [12]:
# Entrenar KNN
knn_scaled = KNeighborsClassifier(n_neighbors=1)
knn_scaled.fit(X_train_scaled, y_train)

y_pred_scaled = knn_scaled.predict(X_test_scaled)
print("\nReporte de clasificación datos normalizados y 3 vecinos:")
print(classification_report(y_test, y_pred_scaled, target_names=["No Pagado", "Pagado"]))

print ("\nMatriz de confusión con datos normalizados y 3 vecinos:\n", confusion_matrix(y_test, y_pred_scaled), "\n")



Reporte de clasificación datos normalizados y 3 vecinos:
              precision    recall  f1-score   support

   No Pagado       0.20      0.19      0.19       883
      Pagado       0.86      0.86      0.86      5090

    accuracy                           0.77      5973
   macro avg       0.53      0.53      0.53      5973
weighted avg       0.76      0.77      0.76      5973


Matriz de confusión con datos normalizados y 3 vecinos:
 [[ 168  715]
 [ 688 4402]] 



In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

# Cargar datos
prestamos_df = pd.read_csv('prestamos_ok.csv')
X = prestamos_df[['loan_amnt','int_rate','annual_inc','dti','purpose_code','grade_code']]
y = prestamos_df['repaid']

# Columnas
num_cols = ['loan_amnt','int_rate','annual_inc','dti']
cat_cols = ['purpose_code','grade_code']

# Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

# Preprocesador: escalar numéricos y one-hot a categóricos
preproc = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
])

# Pipeline
pipe = Pipeline([
    ('prep', preproc),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

# Entrenar y predecir probabilidades
pipe.fit(X_train, y_train)
probs_1 = pipe.predict_proba(X_test)[:, 1]  # probabilidad de Pagado (clase 1)
probs_0 = pipe.predict_proba(X_test)[:, 0]  # probabilidad de No Pagado (clase 0)

# AUC
auc_1 = roc_auc_score(y_test, probs_1)
auc_0 = roc_auc_score(y_test, probs_0)  # AUC para puntajes de la clase 0

print(f"KNN AUC (Pagado / clase 1): {auc_1:.4f}")
print(f"KNN AUC (No Pagado / clase 0): {auc_0:.4f}")

KNN AUC (Pagado / clase 1): 0.5796
KNN AUC (No Pagado / clase 0): 0.4204
